In [16]:
# Make sure that the package can be imported
import sys
from os.path import expanduser, join
home = expanduser("~")
sys.path.append(join(home, 'Repositories/QuREBB'))
from importlib import reload  # Python 3.4+

import lib.states; reload(lib.states)
from lib.states import *

import lib.NQobj as nq; reload(nq)
import lib.quantum_optical_modelling as qom; reload(qom)
import lib.protocol as pc; reload(pc)

import tools.simulation_tools as stools; reload(stools)

from protocols.incoherent_emission import SingleClickProtocol
from protocols.linear_reflection_class import LinearReflection, MidpointReflection

import lib.LBB as lbb; reload(lbb)
import lib.PBB as pbb; reload(pbb)



import numpy as np
import qutip as qt
import scipy.constants as cst
import matplotlib.pyplot as plt

import time, datetime

import h5py as h5
save_path=join(home, 'Repositories/QuREBB/tutorial_simulations/simulation_data/')

import time
from copy import deepcopy

qt.settings.auto_tidyup = False
use_timestamp = False

# Simulating basic params

## Parameters - Reflection based

In [38]:
params_reflection = {
    'f_operation': 0,
    'kappa_in': 22e9/2,
    'kappa_loss': 22e9/2, 
    'gamma': 0.093e9,
    'delta': 0e9,
    'splitting': 1.e9,
    'g': 8.38e9,
    'dim':4,
    'alpha':None,
    'gamma_dephasing': 0.03e9,
    'loss':0.9,
    'link_loss':0.9,
    'insertion_loss': 0.5,
    'dc_rate': 0.000,
}


## Single Photon case

In [39]:
sweep_params_reflection ={
    
    'delta':{
        'range': np.linspace(0e9,100e9, int(50))
    },
    
    'f_operation':{
        'range': np.linspace(-15e9,1e9, int(1000))
    },
}

In [40]:
## Defining protocol classes

lr = LinearReflection('linear_refection', params_reflection, dim=2, photon_names = ['E','L'], sweep_params = sweep_params_reflection)

mr = MidpointReflection('midpoint_reflection', params_reflection, dim=3, photon_names = ['Ea','La','Eb', 'Lb'], sweep_params = sweep_params_reflection)



In [17]:
##saving

if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''

try:
    sim_name= datest+'linear_reflection_sp.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

file open
linear_reflection_sp.h5


In [42]:
try:
    grp_par=datafile.create_group('sweep_params')
    for name in sweep_params:
        grp_par.attrs.create(name, sweep_params_reflection[name]['range'])
    
    grp_data=datafile.create_group('data')
    
    F_lr, R_lr = lr.multiprocess_sweep()
    fr_lr = stools.get_fidelity_rate_curve(F_lr, R_lr)
    
    grp_data.create_dataset('fidelity_map', data=np.array(F_lr))
    grp_data.create_dataset('rate_map', data=np.array(R_lr))
    grp_data.create_dataset('fidelity_curve', data=np.array(fr_lr[0]))
    grp_data.create_dataset('rate_curve', data=np.array(fr_lr[1]))
    grp_data.attrs.update({k: params_reflection[k] for k in params_reflection.keys() - {'alpha'}})
    
finally:
    datafile.close()


Sweep time with multi was 0.503 s


In [13]:
##saving

if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''
    
try:
    sim_name= datest+'midpoint_reflection_sp.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

In [44]:


try:
    grp_par=datafile.create_group('sweep_params')
    for name in sweep_params:
        grp_par.attrs.create(name, sweep_params_reflection[name]['range'])
    
    grp_data=datafile.create_group('data')
    
    F_mr, R_mr = mr.multiprocess_sweep()
    fr_mr = stools.get_fidelity_rate_curve(F_mr, R_mr)
    
    grp_data.create_dataset('fidelity_map', data=np.array(F_mr))
    grp_data.create_dataset('rate_map', data=np.array(R_mr))
    grp_data.create_dataset('fidelity_curve', data=np.array(fr_mr[0]))
    grp_data.create_dataset('rate_curve', data=np.array(fr_mr[1]))
    grp_data.attrs.update({k: params_reflection[k] for k in params_reflection.keys() - {'alpha'}})
    
finally:
    datafile.close()

Sweep time with multi was 0.700 s


### WCS Case

In [16]:
params_wcs = {
    'f_operation': 0,
    'kappa_in': 22e9/2,
    'kappa_loss': 22e9/2, 
    'gamma': 0.093e9,
    'delta': 0e9,
    'splitting': 1.e9,
    'g': 8.38e9,
    'dim':4,
    'alpha':0.001,
    'gamma_dephasing': 0.03e9,
    'loss':0.9,
    'link_loss':0.9,
    'insertion_loss': 0.5,
    'dc_rate': 0.000,
}

In [17]:
pts = 50

sweep_params_wcs ={
    
    'delta':{
        'range': np.linspace(0e9,100e9, int(30))
    },
    
    'f_operation':{
        'range': np.linspace(-15e9,1e9, int(501))
    },
    'alpha':{
        'range': np.linspace( np.sqrt(0.0001) , np.sqrt(2.) , int(10))
    }
}

In [28]:
## Now using protocol class

lr_wcs = LinearReflection('linear_refection', params_wcs, dim=4, photon_names = ['E','L'], sweep_params = sweep_params_wcs)

# mr_wcs = MidpointReflection('midpoint_reflection', params, dim=4, photon_names = ['Ea','La','Eb', 'Lb'], sweep_params = sweep_params)


In [8]:
##saving
if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''

try:
    sim_name= datest+'linear_reflection_wcs.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

file open
linear_reflection_wcs.h5


In [30]:
try:
    grp_par=datafile.create_group('sweep_params')
    for name in sweep_params:
        grp_par.attrs.create(name, sweep_params_wcs[name]['range'])
    
    grp_data=datafile.create_group('data')
    
    F_lr, R_lr = lr_wcs.multiprocess_sweep()
    fr_lr = stools.get_fidelity_rate_curve(F_lr, R_lr)
    
    grp_data.create_dataset('fidelity_map', data=np.array(F_lr))
    grp_data.create_dataset('rate_map', data=np.array(R_lr))
    grp_data.create_dataset('fidelity_curve', data=np.array(fr_lr[0]))
    grp_data.create_dataset('rate_curve', data=np.array(fr_lr[1]))
    grp_data.attrs.update({k: params_wcs[k] for k in params_wcs.keys() - {'alpha'}})
    
finally:
    datafile.close()

Sweep time with multi was 584.363 s


## Emission Based

In [2]:
#Emission based

params_emission = {
    'alpha': 0.1,
    'dim': 3,
    'dc_rate': 0.000,
    'phase': 0.,
    'kappa_in': 240e9,
    'kappa_loss': 90e9,
    'DW': .7,
    'QE': .2,
    'gamma': 0.1e9,
    'gamma_dephasing': 0.03e9,
    'g':6.81e9,
    'link_loss':0.9,
    'insertion_efficiency': 0.5
}

sweep_params_emission ={
    
    'alpha':{
        'range': np.linspace(0.0001, 0.5, 100)
    }
}

In [9]:
##saving
if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''
try:
    sim_name= datest+'midpoint_emission.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

file open
_midpoint_emission.h5


In [4]:


try:
    grp_par=datafile.create_group('sweep_params')
    for name in sweep_params_emission:
        grp_par.attrs.create(name, sweep_params_emission[name]['range'])
    
    grp_data=datafile.create_group('data')
    
    ff,rr = stools.multiprocess_sweep(sweep_params_emission, SingleClickProtocol, params_emission)
    fr = stools.get_fidelity_rate_curve(ff, rr)
    
    grp_data.create_dataset('fidelity_map', data=np.array(ff))
    grp_data.create_dataset('rate_map', data=np.array(rr))
    grp_data.create_dataset('fidelity_curve', data=np.array(fr[0]))
    grp_data.create_dataset('rate_curve', data=np.array(fr[1]))
    grp_data.attrs.update(params_emission)
    
finally:
    datafile.close()

dict_keys(['alpha', 'dim', 'dc_rate', 'phase', 'kappa_in', 'kappa_loss', 'DW', 'QE', 'gamma', 'gamma_dephasing', 'g', 'link_loss', 'insertion_efficiency'])
dict_keys(['alpha', 'dim', 'dc_rate', 'phase', 'kappa_in', 'kappa_loss', 'DW', 'QE', 'gamma', 'gamma_dephasing', 'g', 'link_loss', 'insertion_efficiency'])
dict_keys(['alpha', 'dim', 'dc_rate', 'phase', 'kappa_in', 'kappa_loss', 'DW', 'QE', 'gamma', 'gamma_dephasing', 'g', 'link_loss', 'insertion_efficiency'])
dict_keys(['alpha', 'dim', 'dc_rate', 'phase', 'kappa_in', 'kappa_loss', 'DW', 'QE', 'gamma', 'gamma_dephasing', 'g', 'link_loss', 'insertion_efficiency'])
Sweep time with multi was 0.501 s


# Sweeping Cooperativity

## Reflecion based - Single photon

### Parameters

In [30]:
params = {
    'f_operation': 0,
    'kappa_in': 22e9/2,
    'kappa_loss': 22e9/2, 
    'gamma': 0.093e9,
    'delta': 0e9,
    'splitting': 1.e9,
    'g': 8.38e9,
    #'C':10,
    'dim':4,
    'alpha':None,
    'gamma_dephasing': 0.03e9,
    'loss':0.9,
    'link_loss':0.9,
    'insertion_loss': 0.5,
    'dc_rate': 0.000,
}

pts = 500

sweep_params ={
    
    'delta':{
        'range': np.linspace(-2e9,200e9, int(50))
    },
    
    'f_operation':{
        'range': np.linspace(-30e9,0e9, int(1501))
    },
}




### Cooperativity by increasing kappa

In [29]:
lr = LinearReflection('linear_refection', params, dim=2, photon_names = ['E','L'], sweep_params = sweep_params)


In [10]:
##saving

if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''


try:
    sim_name= datest+'linear_reflection_coop_sweep_kappa.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

file open
linear_reflection_coop_sweep_kappa.h5


In [31]:

c_sweep = np.array([1, 10, 50, 100, 200, 500, 1000]) 

for c in c_sweep:
    kappa_tot = 4*lr.parameters['g']**2/(lr.parameters['gamma']+lr.parameters['gamma_dephasing'])/c
    lr.set_param('kappa_in', kappa_tot/2)
    lr.set_param('kappa_loss', kappa_tot/2)
    
    grp_c = datafile.create_group('c_{}'.format(int(c)))
    try:
        grp_par=grp_c.create_group('sweep_params')
        for name in sweep_params:
            grp_par.attrs.create(name, lr.sweep_params[name]['range'])

        grp_data=grp_c.create_group('data')

        ff,rr = lr.multiprocess_sweep()
        fr = stools.get_fidelity_rate_curve(ff, rr)

        grp_data.create_dataset('fidelity_map', data=np.array(ff))
        grp_data.create_dataset('rate_map', data=np.array(rr))
        grp_data.create_dataset('fidelity_curve', data=np.array(fr[0]))
        grp_data.create_dataset('rate_curve', data=np.array(fr[1]))
        grp_data.attrs.update({k: params[k] for k in params.keys() - {'alpha'}})
    except:
        print('Running C = {} did not work'.format(c))


datafile.close()

Sweep time with multi was 27671.131 s
Sweep time with multi was 26477.413 s
Sweep time with multi was 26660.018 s
Sweep time with multi was 26439.790 s
Sweep time with multi was 26408.322 s


In [8]:
## resetting protocol

lr = lrc.LinearReflection('linear_refection', params, dim=2, photon_names = ['E','L'], sweep_params = sweep_params)


In [11]:
if use_timestamp:
    datest = datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_'
else:
    datest=''
try:
    sim_name= datest+'linear_reflection_coop_sweep_g.h5'
    datafile=h5.File(save_path+ sim_name, 'w')
    print('file open')
    print(sim_name)
except:
    datafile.close()

file open
linear_reflection_coop_sweep_g.h5


In [10]:
ps = {
    "1":
    {"delta": np.linspace(-0.6e10, 0.2e10, 50), 
     "f_operation": np.linspace(-2e9, 4e9, 1501)},
    
    "10":
    {"delta":np.linspace(-1.2e10, 0.4e10, 50), 
     "f_operation":np.linspace(-1e9, 5e9, 1501)},
 
    "50":
    {"delta":np.linspace(-1.5e10, 0.8e10, 50), 
     "f_operation":np.linspace(0e9, 10e9, 1501)},
    
    "100":
    {"delta":np.linspace(-1.5e10, 0.8e10, 50), 
     "f_operation":np.linspace(1e9, 12e9, 1501)},
    
    "200":
    {"delta":np.linspace(-1.8e10, 1.e10, 50), 
     "f_operation":np.linspace(1e9, 25e9, 1501)},
    
    "500":
    {"delta":np.linspace(-4e10, 1.0e10, 50), 
     "f_operation":np.linspace(1e9, 25e9, 1501)},
    
    "1000":
    {"delta":np.linspace(-4e10, 1.0e10, 50), 
     "f_operation":np.linspace(5e9, 25e9, 1501)},
    
#     "2500":
#     {"delta":np.linspace(-5e10, 1.0e10, 50), 
#      "f_operation":np.linspace(5e9, 30e9, 1501)},
    
#     "5000":
#     {"delta":np.linspace(-6e10, 1.0e10, 50), 
#      "f_operation":np.linspace(5e9, 35e9, 1501)},
}

done


In [11]:
c_sweep = np.array([1, 10, 50, 100, 200, 500, 1000]) 

for c in c_sweep:
    g = np.sqrt( c * (lr.parameters['kappa_in']+lr.parameters['kappa_loss']) * (lr.parameters['gamma']+lr.parameters['gamma_dephasing'])/4  )
    lr.set_param('g', g)
    
    lr.set_sweep_param_range('delta', ps[str(c)]["delta"])
    lr.set_sweep_param_range('f_operation',ps[str(c)]["f_operation"])
    
    grp_c = datafile.create_group('c_{}'.format(int(c)))
    try:
        grp_par=grp_c.create_group('sweep_params')
        for name in sweep_params:
            grp_par.attrs.create(name, lr.sweep_params[name]['range'])

        grp_data=grp_c.create_group('data')

        ff,rr = lr.multiprocess_sweep()
        fr = stools.get_fidelity_rate_curve(ff, rr)

        grp_data.create_dataset('fidelity_map', data=np.array(ff))
        grp_data.create_dataset('rate_map', data=np.array(rr))
        grp_data.create_dataset('fidelity_curve', data=np.array(fr[0]))
        grp_data.create_dataset('rate_curve', data=np.array(fr[1]))
        grp_data.attrs.update({k: params[k] for k in params.keys() - {'alpha'}})

    except:
        print('Running C = {} did not work'.format(c))


datafile.close()

Sweep time with multi was 6372.826 s
Sweep time with multi was 6341.079 s
Sweep time with multi was 6383.531 s
Sweep time with multi was 6341.429 s
Sweep time with multi was 6348.810 s
Sweep time with multi was 6389.901 s
Sweep time with multi was 6318.564 s
